In [1]:
import joblib
import numpy as np
import pandas as pd

In [2]:
# data = pd.read_csv('data/final.csv')#合并日期
# data = data.drop(['Latitute','Longtitute'],axis=1)
# data[-3:].to_csv('data/show_data.csv',index=False)

In [3]:
model_name = './models/multi_xgb.model'
model = joblib.load(model_name)
print(model)

MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            enable_categorical=False,
                                            gamma=None, gpu_id=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None,
                                            max_delta_step=None, max_depth=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=Non...
                                            param_grid={'colsample_bytree': 0.5,
                                                        'gamma': 0

C:\Users\Di\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator MultiOutputRegressor from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
data = pd.read_csv('data/show_data.csv',parse_dates=[['Crash Time','DATE']])#合并日期
data = data.drop(['ID'],axis=1)
data.head()

,Crash Time_DATE,Crash Severity,Maximum Injury Severity Reported,Number of Vehicles,Total Nonfatal Injuries,Total Fatal Injuries,Manner of Collision,Vehicle Action Prior to Crash,Vehicle Travel Directions,Road Surface Condition,...,Weather Condition,X Cooordinate,Y Cooordinate,STATION,NAME,PRCP,SNOW,TMAX,TMIN,Ice
0,2021-11-30 23:27:00,Non-fatal injury,Possible Injury (C),2,0,0,Angle,V1: Backing / V2: Travelling straight ahead,V1: N / V2: N,Dry,...,Clear,216659.8125,852126.6934,USC00198368,"NWS BOSTON NORTON, MA US",0,0.0,39,21,True
1,2022-01-23 09:04:00,Property damage only (none injured),No Apparent Injury (O),2,0,0,Single vehicle crash,V1: Travelling straight ahead / V2: Parked,V1: E / V2: E,Dry,...,Clear,217740.1876,854014.6250,USC00198368,"NWS BOSTON NORTON, MA US",0,0.1,26,10,True
2,2022-01-23 14:57:00,Property damage only (none injured),No Apparent Injury (O),2,0,0,Rear-end,V1: Travelling straight ahead / V2: Slowing or...,V1: N / V2: N,Dry,...,Clear,212480.5921,850443.4961,USC00198368,"NWS BOSTON NORTON, MA US",0,0.1,26,10,True
3,2002-01-01 12:00:00,Not Reported,Not reported,2,0,0,Rear-end,V1: Turning left / V2: Travelling straight ahead,V1: N / V2: N,Dry,...,Clear,218441.4064,855385.4998,USC00198368,"NWS BOSTON NORTON, MA US",0,0.0,36,13,True


In [5]:
for df in [data]:
    df['year'] = df['Crash Time_DATE'].dt.year
    df['month'] = df['Crash Time_DATE'].dt.month
    df['day'] = df['Crash Time_DATE'].dt.day
    df['hour'] = df['Crash Time_DATE'].dt.hour

    df['hour_category'] = np.nan
    df['season'] = np.nan
    df['week'] = np.nan
    
    df.loc[(df.hour <= 9)&(df.hour >= 7), 'hour_category'] = "rush"
    df.loc[(df.hour <= 19)&(df.hour >= 16), 'hour_category'] = "rush"
    df.loc[(df.hour < 16)&(df.hour > 9), 'hour_category'] = "day"
    df.loc[(df.hour < 7)|(df.hour > 19), 'hour_category'] = "night"
    
    df.loc[(df.month <= 3)&(df.month >= 1), 'season'] = 1
    df.loc[(df.month <= 6)&(df.month >= 4), 'season'] = 2
    df.loc[(df.month <=9)&(df.month >= 7), 'season'] = 3
    df.loc[(df.month <=12)&(df.month >=10), 'season'] = 4
    
    df.loc[(df.day <= 7)&(df.day >= 1), 'week'] = 1
    df.loc[(df.day <= 14)&(df.day >= 8), 'week'] = 2
    df.loc[(df.day <=21)&(df.day >= 15), 'week'] = 3
    df.loc[(df.day >=22), 'week'] = 4

In [6]:
Severity = pd.get_dummies(data['Crash Severity'], prefix='Severity', prefix_sep='_')

Injury_Reported = pd.get_dummies(data['Maximum Injury Severity Reported'], prefix='Injury_Reported', prefix_sep='_')

Manner = pd.get_dummies(data['Manner of Collision'], prefix='Manner', prefix_sep='_')

Surface_Condition = pd.get_dummies(data['Road Surface Condition'], prefix='Condition', prefix_sep='_')

Ambient_Light = pd.get_dummies(data['Ambient Light'], prefix='Light', prefix_sep='_')

Weather_Condition = pd.get_dummies(data['Weather Condition'], prefix='Weather', prefix_sep='_')

Ice = pd.get_dummies(data['Ice'], prefix='Ice', prefix_sep='_')

Hour = pd.get_dummies(data['hour_category'], prefix='Hour', prefix_sep='_')

Season = pd.get_dummies(data['season'], prefix='season', prefix_sep='_')

Week = pd.get_dummies(data['week'], prefix='week', prefix_sep='_')


In [7]:
Data_Master = pd.concat([data,
                          Severity,
                          Injury_Reported,
                          Manner,
                          Surface_Condition,
                          Ambient_Light,
                          Weather_Condition,
                          Ice,
                          Hour,
                          Season,
                          Week
                         ], axis=1)

In [8]:
data = Data_Master.drop(['Crash Severity','Maximum Injury Severity Reported',
                                  'Manner of Collision','Road Surface Condition',
                                  'Ambient Light','Weather Condition','Ice','hour_category','Crash Time_DATE'], axis = 1)

In [9]:
file_obj = open("data/attributes.txt")
all_lines = file_obj.readlines()
columns = []
for line in all_lines:
    columns.append(line.strip())
columns.remove('Longtitute')
columns.remove('Latitute')

In [10]:
for column in columns:
    if column not in data.columns:
        data[column] = 0

In [11]:
results = model.predict(data)

In [12]:
results

array([[ 42.01505 , -71.46833 ],
       [ 41.621456, -71.19471 ],
       [ 41.587578, -71.174805],
       [ 38.530342, -71.797195]], dtype=float32)

In [13]:
# data_item = {}
# Crash_Time_DATE= input("input: Crash Time_DATE: ")
# Crash_Severity = input("input: Crash_Severity: ")
# Maximum_Injury_Severity_Reported = input("input: Maximum Injury Severity Reported: ")
# Number_of_Vehicles = input("input: Number_of_Vehicles: ")
# Total_Fatal_Injuries = input("input: Total_Fatal_Injuries: ")
# Total_Nonfatal_Injuries = input("input: Total_Nonfatal_Injuries: ")
# Manner_of_Collision = input("input: Manner_of_Collision: ")
# Road_Surface_Condition = input("input: Road_Surface_Condition: ")
# Ambient_Light = input("input: Ambient_Light: ")
# Weather_Condition = input("input: Weather_Condition: ")
# PRCP = input("input: PRCP: ")
# SNOW = input("input: SNOW: ")
# TMAX = input("input: TMAX: ")
# TMIN = input("input: TMIN: ")
# Ice = input("input: Ice: ")